In [1]:
import time
from datetime import datetime as dt

from google.cloud import bigtable
from google.cloud.bigtable import column_family, row_filters
from google.cloud.bigtable.row_set import RowSet
import google.cloud.bigtable.row_filters as row_filters
import pandas as pd

/usr/lib/python3.10/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 2.0.5-build-libtorrent-rasterbar-src-libtorrent-rasterbar-2.0.5-bindings-python is an invalid version and will not be supported in a future release
  warnings.warn(


### Conection

In [2]:
INSTANCE_ID = 'bigtable-1'
TABLE_ID = 'Ambiente{}'.format(time.time())
client = bigtable.Client.from_service_account_json('../unbosque-service-account.json', admin = True)
instance = client.instance(  INSTANCE_ID)

In [3]:
p2_weatherdata = pd.read_csv("../data/Plant_2_Weather_Sensor_Data.csv")

### Creating Table plant 2 plant2 weatherdata

In [4]:
"""Creating Table plant 1 generation data csv"""
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

Creating the Ambiente1649644933.4960532 table


In [5]:
print('Creating all family columns')
mediciones_cfId = 'Mediciones'
mediciones_cf = table.column_family(mediciones_cfId)
mediciones_cf.create()


Creating all family columns


### Inserting data into table plant 1 generation data 

In [6]:
dt = dt.utcnow()
rows = []

json = p2_weatherdata.to_dict(orient='records')

print('Writing rows')

for plant in json:
    row_key = 'plant2#{}'.format(plant['DATE_TIME']).encode()
    row = table.direct_row(row_key)
    row.set_cell(mediciones_cfId, 'temp_ambient'.encode(), str(plant['AMBIENT_TEMPERATURE']) , timestamp=dt)
    row.set_cell(mediciones_cfId, 'temp_module'.encode(), str(plant['MODULE_TEMPERATURE']) , timestamp=dt)
    row.set_cell(mediciones_cfId, 'irradiation'.encode(), str(plant['IRRADIATION']) , timestamp=dt)
    rows.append(row)
table.mutate_rows(rows)
  


Writing rows


[,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 

In [7]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode("utf-8")))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = (
                    " [{}]".format(",".join(cell.labels)) if len(cell.labels) else ""
                )
                print(
                    "\t{}: {} @{}{}".format(
                        col.decode("utf-8"),
                        cell.value.decode("utf-8"),
                        cell.timestamp,
                        labels,
                    )
                )
    print("")

In [8]:
row_set = RowSet()
row_set.add_row_range_from_keys(
    start_key=b"plant2#2020-05-15 06:00:00", end_key=b"plant2#2020-05-15 12:00:00"
)
col_filter = row_filters.ColumnQualifierRegexFilter(b"temp_ambient")
rows = table.read_rows(row_set=row_set,filter_=col_filter )
for row in rows:
    print_row(row)

Reading data for plant2#2020-05-15 06:00:00:
Column Family Mediciones
	temp_ambient: 24.74278656666667 @2022-04-11 02:43:43.720000+00:00

Reading data for plant2#2020-05-15 06:15:00:
Column Family Mediciones
	temp_ambient: 24.76391089655172 @2022-04-11 02:43:43.720000+00:00

Reading data for plant2#2020-05-15 06:30:00:
Column Family Mediciones
	temp_ambient: 24.7863229 @2022-04-11 02:43:43.720000+00:00

Reading data for plant2#2020-05-15 06:45:00:
Column Family Mediciones
	temp_ambient: 25.04824510344827 @2022-04-11 02:43:43.720000+00:00

Reading data for plant2#2020-05-15 07:00:00:
Column Family Mediciones
	temp_ambient: 25.6732025 @2022-04-11 02:43:43.720000+00:00

Reading data for plant2#2020-05-15 07:15:00:
Column Family Mediciones
	temp_ambient: 26.076818 @2022-04-11 02:43:43.720000+00:00

Reading data for plant2#2020-05-15 07:30:00:
Column Family Mediciones
	temp_ambient: 26.424239066666665 @2022-04-11 02:43:43.720000+00:00

Reading data for plant2#2020-05-15 07:45:00:
Column Fam